In [ ]:
import cv2  #imported necessary libraries 
import numpy as np
import face_recognition
import os
from datetime import datetime
path='C:\\Users\\gupta\\source\\repos\\Face_Recognition_Attendance\\TRAINING'# path where images are stored inside VS repos
images=[]
className=[]
myList=os.listdir(path)
print(myList)
for cl in myList:
    curImg=cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    className.append(os.path.splitext(cl)[0])
print(className)
def faceEncodings(images):
    encodeList = []
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = faceEncodings(images)
print('Encoding Complete')

def attendance(name):

    with open('attendance.csv','r+') as f:#attendance sheet name
        myDataList =f.readlines()
        nameList=[]
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])

        if name not in nameList:
            time_now =datetime.now()
            tStr = time_now.strftime('%H:%M:%S')
            dStr= time_now.strftime('%d/%m/%Y')
            f.writelines(f'{name},{tStr},{dStr}')
           

cap=cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS=cv2.resize(img,(0,0),None,0.25,0.25)
    imgS =cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB)
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame=face_recognition.face_encodings(imgS,facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis =face_recognition.face_distance(encodeListKnown,encodeFace)
        print(faceDis)
        matchIndex =np.argmin(faceDis)

        if matches[matchIndex]:
            name=className[matchIndex].upper()
            
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 =y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img, name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            attendance(name)
    cv2.imshow("Camera", img)
    if cv2.waitKey(0) == 13:
        break
cap.release()
cv2.destroyAllWindows()